In [21]:
import numpy as np
from itertools import groupby

#### Вхідні дані

In [22]:
alternatives = [[ 7, 7, 9, 10, 10, 5, 2, 3, 5, 3, 9, 1],
                [ 4, 9, 8, 6, 6, 9, 3, 10, 10, 3, 2, 6],
                [ 2, 8, 8, 1, 1, 4, 2, 9, 1, 10, 3, 1],
                [ 4, 10, 9, 4, 8, 5, 2, 8, 9, 10, 8, 5],
                [ 4, 5, 5, 10, 1, 9, 5, 3, 1, 4, 4, 7],
                [ 7, 7, 5, 3, 10, 3, 2, 7, 1, 2, 3, 1],
                [ 3, 7, 7, 8, 9, 7, 2, 10, 2, 1, 5, 4],
                [ 4, 1, 10, 3, 7, 1, 3, 2, 2, 5, 7, 4],
                [ 10, 2, 5, 1, 9, 2, 9, 3, 6, 6, 4, 8],
                [ 10, 6, 4, 4, 7, 9, 3, 6, 4, 5, 7, 9],
                [ 4, 7, 7, 3, 1, 7, 6, 3, 3, 5, 7, 10],
                [ 10, 4, 1, 9, 2, 4, 5, 1, 2, 3, 1, 10],
                [ 7, 10, 7, 10, 9, 3, 4, 1, 2, 9, 4, 6],
                [ 9, 4, 2, 6, 2, 10, 10, 6, 5, 3, 9, 6],
                [ 6, 5, 2, 2, 6, 1, 6, 2, 1, 7, 3, 8]]

weights = [ 4, 1, 5, 6, 4, 2, 1, 2, 9, 2, 6, 1]

c = 0.660
d = 0.461

#### ELECTRE I

In [23]:
#
def count_diff_ai_bi(alternatives_matrix, weights_array):
    alt = np.array(alternatives_matrix)
    weights = np.array(weights_array)
    diffs_ai_bi = []
    for i in range (0, len(alt[0,:])):
        diffs_ai_bi.append(weights[i]*(np.amax(alt[:, i])-np.amin(alt[:,i])))
    return diffs_ai_bi

#count_diff_ai_bi(alternatives, weights)

def sum_ij(a1, a2, weights_array):
    result_sum = 0
    for i in range(0, len(a1)):
        if a1[i] >= a2[i]:
            result_sum += weights_array[i]
    return result_sum

def c_matrix(alternatives_matrix, weights_array):
    result_matrix = [[0]*15 for i in range(15)]
    weights_sum = np.sum(weights_array)
    for i in range(0, 15):
        for j in range(0, 15):
            if i != j:
                result_matrix[i][j] = sum_ij(alternatives_matrix[i], alternatives_matrix[j], weights_array)/weights_sum
    f = open("с_matrix.txt", "w")
    for i in range(0,len(result_matrix)):
        for j in range(0,len(result_matrix)):
            f.write(" {:.3f} ".format(result_matrix[i][j]))
        f.write('\n')
    f.close()
    return result_matrix
#c_matrix(alternatives, weights)

def d_ij(a1, a2, weights_array, diffs_ai_bi):
    diffs = []
    selected_diffs_ai_bi = []
    for i in range(0, len(a1)):
        if a1[i] < a2[i]:
            diffs.append(weights_array[i]*(a2[i]-a1[i]))
            selected_diffs_ai_bi.append(diffs_ai_bi[i])   
    try:
        return np.amax(diffs)/np.amax(selected_diffs_ai_bi)
    except:
        return 0

def d_matrix(alternatives_matrix, weights_array):
    result_matrix = [[1]*15 for i in range(15)]
    diffs_ai_bi = count_diff_ai_bi(alternatives_matrix, weights_array)
    for i in range(0, 15):
        for j in range(0, 15):
            if i != j:
                result_matrix[i][j] = d_ij(alternatives_matrix[i], alternatives_matrix[j], weights_array, diffs_ai_bi)
    f = open("d_matrix.txt", "w")
    for i in range(0,len(result_matrix)):
        for j in range(0,len(result_matrix)):
            f.write(" {:.3f} ".format(result_matrix[i][j]))
        f.write('\n')
    f.close()
    return result_matrix

#d_matrix(alternatives, weights)
#
def electre_I(alternatives_matrix, weights_array, c, d):
    C_matrix = np.array(c_matrix(alternatives_matrix, weights_array))
    D_matrix = np.array(d_matrix(alternatives_matrix, weights_array))
    R = []
    X = []
    for i in range(0, len(C_matrix)):
        for j in range(0, len(C_matrix)):
            if i != j and C_matrix[i][j] >= c and D_matrix[i][j] <= d:
                R.append([i, j])
    for i in range(0, 15):
        flag = True
        for j in range(0, len(R)):
            if i in R[j]:
                flag = False
                break
        if flag:
            X.append(i)
    for i in range(0, len(R)):
        pair = R[i]
        x_candidate = pair[0]
        flag = True
        for j in range(0, len(R)):
            pair_compare = R[j]
            if pair_compare[1] == x_candidate:
                flag = False
                break
        if flag:
            X.append(x_candidate)
    X.sort()
    X_result = [el for el, _ in groupby(X)]
    for i in range(0, len(X_result)):
        X_result[i] = X_result[i] + 1
    return X_result

electre_I(alternatives, weights, c, d)
                

[1, 2, 4, 9, 13]